## Main code to analyse preservation trends in bar packages. 
Inputs = CSVs with bar stats, and various input arrays for package statistics

In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from copy import deepcopy
import itertools
from matplotlib import cm
from pylab import *


import os
import seaborn as sns
from os import listdir
from os.path import isfile, join
import scipy.stats as stats

In [3]:
## Set Color maps for plots and plot aesthetics

coher_cols = ['#276419', '#9ccf64', '#f7f7f7', '#e896c4', '#8e0152'] ##PiYG 5 steps
coher_cols_seq = ['#d8ffff', '#85d9f0', '#2cb0e0', '#0083c6', '#0057a7'] #light blue to blue
# pres_cols = ['xkcd:pink', 'xkcd:lavender', 'xkcd:light grey'] #pink = FP, lav = PP, blue = TR
pres_cols = ['xkcd:primary blue', 'xkcd:soft blue', 'xkcd:pink red']
proc_cols = ['#ff8500', '#ffca20', '#ffffe0', '#bf74e7', '#8044da'] #(orange to purple) agg to lat acc
pres_palette = sns.color_palette(pres_cols)
coh_palette = sns.color_palette(coher_cols_seq)
proc_palette = sns.color_palette(proc_cols)

font = {'family' : 'Helvetica',
        'weight' : 'normal',
        'size'   : 8}

mpl.rc('font', **font)

Import files

In [5]:
filenm = 'agubh2-100m'
xlsxname = filenm+'.xlsx'
savefilesto = '/Volumes/SAF_Data/bar-manuscript_sum22/data-interp/'
mypath = f'/Volumes/SAF_Data/NAYS2DH_files/Data/BarCSVs/concat-files/{xlsxname}'
allbars = pd.read_excel(mypath, header=0, usecols = 'A:BF') ## change if you add more columns to the raw dataset
eqt = 65 ## time for the bed to equilibrate

## import all additional datasets
## array of number of active channels
active_channels = np.load('/Volumes/SAF_Data/NAYS2DH_files/Data/nparrays/agubh2_active_channels-ud.npy') 

## array of wetted width
wetted_width = np.load('/Volumes/SAF_Data/NAYS2DH_files/Data/nparrays/agubh2WWmaster-ud.npy') 
wetted_width[:eqt, :] = np.nan 

##csv of skewness values of centroids
skewness = pd.read_csv('/Volumes/SAF_Data/NAYS2DH_files/Data/BarCSVs/concat-files/CentroidTrajectories_Skew.csv') 
skewness['File'] = skewness['File'].str.replace('.npy', '')

## array of thickness measurements of packages like Sinead
cvs = pd.read_csv(f'/Volumes/SAF_Data/NAYS2DH_files/Data/BarCSVs/concat-files/LysterthicknessCVs.csv') 
cvs['File'] = cvs['File'].str.replace('.npy', '')

for col, dat in cvs.iloc[:, 6:].iteritems():
    cvs[col] = cvs[col].str.replace('[', '', regex = False)
    cvs[col] = cvs[col].str.replace(']', '', regex = False)
    cvs[col] = cvs[col].astype(float)

/var/folders/yg/89nlrh5x2xzgdlgr2bczm4_m0000gn/T/ipykernel_5872/3141334978.py:18: FutureWarning: The default value of regex will change from True to False in a future version.
  skewness['File'] = skewness['File'].str.replace('.npy', '')
/var/folders/yg/89nlrh5x2xzgdlgr2bczm4_m0000gn/T/ipykernel_5872/3141334978.py:22: FutureWarning: The default value of regex will change from True to False in a future version.
  cvs['File'] = cvs['File'].str.replace('.npy', '')


### Add extra ratios to allbars dataframe


In [6]:
allbars['CVFlowDepth'] = allbars['SDFlowDepth']/allbars['MeanFlowDepth']
allbars['CVVelocity'] = allbars['SDVelocity']/allbars['MeanVelocity']
allbars['Skew Abs'] = abs(allbars['Angle Skewness'])


# df.loc[ df[“column_name”] == “some_value”, “column_name”] = “value”
allbars['TempAgg'] = allbars['MedClinoHt']/allbars['MaxClinoHt']
allbars['Skew Binned'] = allbars['Skew Abs']
allbars.loc[np.logical_or(allbars['Skew Abs'] > 1, allbars['Skew Abs'] < 3), 'Skew Binned'] = 1
allbars.loc[allbars['Skew Abs'] < 1, 'Skew Binned'] = 0
allbars.loc[allbars['Skew Abs'] > 3, 'Skew Binned'] = 2

allbars['Co Binned'] = allbars['Skew Abs']
allbars.loc[np.logical_or(allbars['CoherenceVal'] > 2, allbars['CoherenceVal'] <= 3), 'Co Binned'] = 1
allbars.loc[allbars['CoherenceVal'] <= 2, 'Co Binned'] = 0

allbars.loc[allbars['CoherenceVal'] > 3, 'Co Binned'] = 2

allbars['BSR Binned'] = allbars['Basal Surf Relief']
allbars.loc[np.logical_or(allbars['Basal Surf Relief'] > 0.5, allbars['Basal Surf Relief'] < 1), 'BSR Binned'] = 1
allbars.loc[allbars['Basal Surf Relief'] < 0.5, 'BSR Binned'] = 0
allbars.loc[allbars['Basal Surf Relief'] > 1, 'BSR Binned'] = 2

## recalc facies
d50 = 0.00031
# chezy_rough = 18*(np.log10(4*allbars['MedFlowDepth']/d50)) #i think this is log10, else they would've put ln no?

med_mmp =(1000*(allbars['MedVelocity']**2))/(1650*((18*(np.log10(4*allbars['MedFlowDepth']/d50)))**2)*d50)##### modified mobility parameter
allbars['MeanFacies'] =(1000*(allbars['MeanVelocity']**2))/(1650*((18*(np.log10(4*allbars['MeanFlowDepth']/d50)))**2)*d50)##### modified mobility parameter
allbars['MedFacies'] =(1000*(allbars['MedVelocity']**2))/(1650*((18*(np.log10(4*allbars['MedFlowDepth']/d50)))**2)*d50)##### modified mobility parameter

allbars['BinFacMean'] = allbars['MeanFacies']
allbars.loc[allbars['MeanFacies'] < 0.17, 'BinFacMean'] = 'LB'
allbars.loc[allbars['MeanFacies'] > 0.17, 'BinFacMean'] = 'UB'

allbars['BarHeight/MaxFD'] = allbars['BarHeight']/allbars['MaxFlowDepth']
allbars['MaxClino/MaxFD'] = allbars['MaxClinoHt']/allbars['MaxFlowDepth']
allbars['MaxClino/MedFD'] = allbars['MaxClinoHt']/allbars['MedFlowDepth']
allbars['MedClino/MedFD'] = allbars['MedClinoHt']/allbars['MedFlowDepth']

allbars['BFscaling'] = allbars['MaxClinoHt']/1.5
allbars['BL/CL'] = allbars['BarWidth']/allbars['MaxClinoWt']
allbars['ReconFlowDepth'] = allbars['MaxClinoHt']/0.7 ## I think its 0.7 

## Find the proportion of upper bar facies in each bar package

allbars['UBar'] = np.nan
allbars['mmpmean'] = np.nan
allbars['FaciesCV'] = np.nan

arrayfolder = 'agubh2-100m-mmp'
# init = '/Volumes/SAF_Data/NAYS2DH_files/Data/nparrays/barpkg-arrays/Summer-bpkg-redo/JULY142022'
init = f'/Volumes/SAF_Data/NAYS2DH_files/Data/nparrays/barpkg-arrays/{arrayfolder}'

for root, dirs, files in os.walk(init):
    for file in files:
        arr = np.load(os.path.join(root, file), allow_pickle = True)
        arrnm = file[:-4]
        mmp = arr[:, :, -1] ## the arrays already have the mmp values in them
        mmpmean = np.nanmean(mmp)
        mmpcv = np.nanstd(mmp)/np.nanmean(mmp)

        tot = np.count_nonzero(~np.isnan(mmp))
        pub = (mmp <= 0.15).sum()/tot
        plb = (mmp > 0.15).sum()/tot
        allbars.loc[allbars['array name']==arrnm, 'UBar'] = pub*100
        allbars.loc[allbars['array name']==arrnm, 'mmpmean'] = mmpmean
        allbars.loc[allbars['array name']==arrnm, 'FaciesCV'] = mmpcv

allbars = pd.merge(allbars, skewness, left_on = 'array name', right_on = 'File')
allbars = pd.merge(allbars, cvs, left_on = 'array name', right_on = 'File')

allbars['CVClinoWt'] = allbars['SDClinoWt']/allbars['MeanClinoWt']

In [7]:
allbars.columns

Index(['BarName', 'array name', 'iloc', 'PreservChar', 'PresNum',
       'CoherenceVal', 'StartTime', 'EndTime', 'ElapsedTime', 'LeftEdge',
       'RightEdge', 'BarWidth', 'BarHeight', 'BarAspect', 'BarArea',
       'Section Area', 'ChannelProp', 'Angle Skewness', 'Basal Surf Relief',
       'Mean dzdx', 'Median dzdx', 'MinClinoHt', 'MaxClinoHt', 'ModeClinoHt',
       'MeanClinoHt', 'SDClinoHt', 'MedClinoHt', 'MinClinoWt', 'MaxClinoWt',
       'ModeClinoWt', 'MeanClinoWt', 'SDClinoWt', 'MedClinoWt', 'MinClinoICD',
       'MaxClinoICD', 'ModeClinoICD', 'MeanClinoICD', 'SDClinoICD',
       'MedClinoICD', 'MinFlowDepth', 'MaxFlowDepth', 'ModeFlowDepth',
       'MeanFlowDepth', 'SDFlowDepth', 'MedFlowDepth', 'MinShear', 'MaxShear',
       'ModeShear', 'MeanShear', 'SDShear', 'MedShear', 'MinVelocity',
       'MaxVelocity', 'ModeVelocity', 'MeanVelocity', 'SDVelocity',
       'MedVelocity', 'process interp', 'CVFlowDepth', 'CVVelocity',
       'Skew Abs', 'TempAgg', 'Skew Binned', 'Co Binne